# Creazione e analisi della rete User-User basata sui followers

In [1]:
import pymongo
from pymongo import MongoClient
import json
import sys

import pygraphviz as pgv
import networkx as nx
from networkx.drawing.nx_agraph import read_dot
from networkx.classes.reportviews import NodeView, EdgeView, DegreeView

import matplotlib.pyplot as plt

client = MongoClient('mongodb://127.0.0.1:27017')
db = client.sample
cutfs = db.user_team_first_shot.find()


### Insieme di utenti di riferimento

In [1]:
def get_users():
    U = []
    for user in cutfs:
        if user['user'] not in U:
            U.append(user['user'])

### Trova l'insieme F dei followers di un utente (tra gli utenti dell'insieme di riferimento, U)

In [17]:
def find_followers(user,U):
    user_folls = []
    
    try:
        folls = db.followers.find({'username': user})[0]['followers']
    except:
        print 'Error - user not in followers dataset'
        folls = []
        
    for f in folls:
        if f['follower_user'] in U:
            user_folls.append(f['follower_user'])
    
            
    return user_folls

# Costruisci la rete

In [18]:
def create_net(U):

    g = pgv.AGraph(directed=True)

    for k in range(0, len(U)):
        g.add_node(U[k])
        u_folls = find_followers(U[k],U)
        for f in u_folls:      
            g.add_node(f) 
            # f segue u quindi creo l'arco (f->u)
            g.add_edge(f,U[k])

        print k

    g.write('userFollowersGraph_test.dot')

# Analisi della rete - User Features

In [2]:
G = read_dot('userFollowersGraph.dot')

In [3]:
def get_in_degree(user):
    in_deg = G.in_degree(user)
    return in_deg

In [65]:
def get_out_degree(user):
    out_deg = G.out_degree(user)
    return out_deg

In [72]:
def get_degree_centrality(user):
    return nx.degree_centrality(G)[user]

In [78]:
def get_betweenness_centrality(user):
    return nx.betweenness_centrality(G)[user]

In [3]:
#%%time
#get_betweenness_centrality('Mikmiria')

In [ ]:
def get_closeness_centrality(user):
    return nx.closeness_centrality(G)[user]

In [75]:
def get_eigenvector_centrality(user):
    return nx.eigenvector_centrality(G)[user]

CPU times: user 28.6 s, sys: 580 ms, total: 29.2 s
Wall time: 28.6 s


0.00015390305660380578

In [4]:
def get_pagerank(user):
    return nx.pagerank(G)[user]